In [1]:
import time
import torch
import ttnn
from llama2.model import ModelArgs
import random
from models.utility_functions import comp_pcc

random.seed(42)
torch.manual_seed(42)

2025-02-14 15:31:40.291 | DEBUG    | ttnn:<module>:82 - Initial ttnn.CONFIG:
Config{cache_path=/home/bach/.cache/ttnn,model_cache_path=/home/bach/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}
2025-02-14 15:31:40.366 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.pearson_correlation_coefficient be migrated to C++?
2025-02-14 15:31:40.367 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.Conv1d be migrated to C++?
2025-02-14 15:31:40.368 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.conv2d be migrated to C++?
2025-02-14 15:31:40.369 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.unsq

In [2]:
TEST = {
    "Attention": True,
    "RMSNorm": True,
    "FeedForward": True,
    "TransformerBlock": True,
    "Transformer": True,
    "Generation": True
}
TEST_LAYER_NUM = 0

In [3]:
from llama2.tokenizer import Tokenizer

# Load checkpoint dict
checkpoint_dict = torch.load("llama2/configs/stories42M.pt")
enc = Tokenizer(tokenizel_path="./llama2/tokenizerM.model")

model_args = checkpoint_dict['model_args']
state_dict = checkpoint_dict['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

args = ModelArgs(**model_args)
print(args)

ModelArgs(dim=512, n_layers=8, n_heads=8, n_kv_heads=8, vocab_size=32000, hidden_dim=None, multiple_of=32, norm_eps=1e-05, max_seq_len=1024, dropout=0.5)


In [4]:
batch_size = 1
tokens_num = args.max_seq_len
device_id=0
device = ttnn.open_device(device_id=device_id)


                 Device | INFO     | Opening user mode device driver

2025-02-14 15:31:40.584 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.30.0, IOMMU: disabled
2025-02-14 15:31:40.584 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-02-14 15:31:40.584 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0


In [5]:
from llama2.model import precompute_freqs_cis
torch_freqs_cos, torch_freqs_sin = precompute_freqs_cis(args.dim // args.n_heads, args.max_seq_len)
freqs_cos = torch_freqs_cos.bfloat16()
freqs_sin = torch_freqs_sin.bfloat16()

In [6]:
# torch.set_printoptions(linewidth=100, precision=3,profile='full')
torch.set_printoptions(profile='short')
def check_close(torch_tensor, ttnn_tensor, atol=0.01):
    ttnn_tensor = torch.Tensor(ttnn.to_torch(ttnn_tensor))
    assert ttnn_tensor.shape == torch_tensor.shape, f"TTNN shape: {ttnn_tensor.shape}, Torch shape: {torch_tensor.shape}"
    # equals = torch.sum(torch.isclose(torch_tensor, ttnn_tensor, atol=atol))
    # perc = equals/torch_tensor.numel() * 100
    # print(f"Close values: {perc:.3f}% ({torch_tensor.numel()})")
    pcc = comp_pcc(torch_tensor, ttnn_tensor)[1]
    print(f"PCC: {pcc:.5f}")
    return pcc

### Compare Attention layer


In [7]:
from llama2.model import Attention as torchAttention
# from ttllama2 import Attention
from ttllama2.attention_faster import Attention

if TEST['Attention']:
    ttnn.close_device(device)
    device = ttnn.open_device(device_id=device_id)
    device.enable_program_cache()
    x_torch = torch.rand((batch_size, tokens_num, args.dim), dtype=torch.bfloat16) 

    x = ttnn.from_torch(
        x_torch,
        layout=ttnn.TILE_LAYOUT, 
        device=device,
        dtype=ttnn.bfloat16
    )

    start = time.time()
    attention = Attention(args, state_dict, TEST_LAYER_NUM, device)
    print(f"Attention init: {time.time() - start:.3f}")

    print("Attention forward started...")
    
    start = time.time()
    o_attention = attention.forward(x, freqs_cos, freqs_sin)

    print("\nAttention forward endend...")

    print(f"TTNN forward: {time.time() - start:.3f}")
    
    start = time.time()
    torch_attention = torchAttention(args, state_dict, TEST_LAYER_NUM)
    print(f"Torch forward: {time.time() - start:.3f}")
    to_attention = torch_attention.forward(x_torch, torch_freqs_cos, torch_freqs_sin)
    
    check_close(to_attention, o_attention)

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
                  Metal | INFO     | Enabling program cache on device 0
Attention init: 0.679
Attention forward started...
Reshapes: 0.373
Rotatory emb: 0.029
Output reshape: 2.447

Attention forward endend...
TTNN forward: 3.474
Torch forward: 0.010
PCC: 0.99181


In [8]:
from llama2.model import Attention as torchAttention
# from ttllama2 import Attention
from ttllama2.attention_faster import Attention

if False:
    ttnn.close_device(device)
    device = ttnn.open_device(device_id=device_id)
    device.enable_program_cache()

    attention = Attention(args, state_dict, TEST_LAYER_NUM, device)
    torch_attention = torchAttention(args, state_dict, TEST_LAYER_NUM)

    n_exec = 100
    tot_time_ttnn = 0
    tot_time_torch = 0
    avg_pcc = 0
    for _ in range(n_exec):
        x_torch = torch.rand((batch_size, tokens_num, args.dim), dtype=torch.bfloat16) 

        x = ttnn.from_torch(
            x_torch,
            layout=ttnn.TILE_LAYOUT, 
            device=device,
            dtype=ttnn.bfloat16
        )
        
        start = time.time()
        o_attention = attention.forward(x, freqs_cos, freqs_sin)
        tot_time_ttnn += time.time() - start

        start = time.time()
        to_attention = torch_attention.forward(x_torch, torch_freqs_cos, torch_freqs_sin)
        tot_time_torch += time.time() - start

        avg_pcc += check_close(to_attention, o_attention)

    avg_time_ttn = tot_time_ttnn / n_exec
    avg_time_torch = tot_time_torch / n_exec
    avg_pcc /= n_exec

    print(f"TTNN forward: {avg_time_ttn}")
    print(f"Torch forward: {avg_time_torch}")
    print(f"Avg PCC: {avg_pcc}")
    
    check_close(to_attention, o_attention)

### Compare FeedForward layer

In [9]:
from llama2.model import FeedForward as torchFeedForward
from ttllama2 import FeedForward

if TEST['FeedForward']:

    ttnn.close_device(device)
    device = ttnn.open_device(device_id=device_id)
    device.enable_program_cache()

    x_torch = torch.rand((batch_size, tokens_num, args.dim), dtype=torch.bfloat16) 
    x = ttnn.from_torch(
        x_torch,
        layout=ttnn.TILE_LAYOUT, 
        device=device,
        dtype=ttnn.bfloat16
    )
    tt_ff = FeedForward(args.dim, args.hidden_dim, args.multiple_of, TEST_LAYER_NUM, state_dict, device)
    torch_ff = torchFeedForward(args.dim, args.hidden_dim, args.multiple_of, 0, state_dict, TEST_LAYER_NUM)
    
    start = time.time()
    o_ff = tt_ff.forward(x)
    print("Transformer:")
    print(f"ttnn: {time.time() - start:.3f}")
    start = time.time()
    to_ff = torch_ff.forward(x_torch)
    print(f"torch: {time.time() - start:.3f}")

    check_close(to_ff, o_ff)

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
Transformer:
ttnn: 2.278
torch: 1.536
PCC: 0.86917


### Compare Transformer Block


In [10]:
from llama2.model import TransformerBlock as torchTransformerBlock
from ttllama2 import TransformerBlock 


if TEST['TransformerBlock']:
    ttnn.close_device(device)
    device = ttnn.open_device(device_id=device_id)
    device.enable_program_cache()
    
    x_torch = torch.rand((batch_size, tokens_num, args.dim), dtype=torch.bfloat16)
        
    x = ttnn.from_torch(
        x_torch,
        device=device,
        dtype=ttnn.bfloat16
    )
    x = ttnn.to_layout(x, ttnn.TILE_LAYOUT)

    tt_tb = TransformerBlock(TEST_LAYER_NUM, args, state_dict, device)
    torch_tb = torchTransformerBlock(TEST_LAYER_NUM, args, state_dict)
    _, seq_len, _ = x_torch.shape

    start = time.time()
    o_tb = tt_tb.forward(x, freqs_cos[:seq_len], freqs_sin[:seq_len])
    print("TransformerBlock:")
    print(f"ttnn: {time.time() - start:.3f}")
    start = time.time()
    to_tb = torch_tb.forward(x_torch, freqs_cos[:seq_len], freqs_sin[:seq_len])
    print(f"torch: {time.time() - start:.3f}")
    check_close(to_tb.bfloat16(), o_tb) 

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
Reshapes: 0.046
Rotatory emb: 0.029
Output reshape: 0.077
TransformerBlock:
ttnn: 3.077
torch: 3.371
PCC: 0.59973


### Compare Transformer

Close values drops hard here...

In [11]:
from llama2.model import Transformer as torchTransformer
from ttllama2 import Transformer
from llama2.tokenizer import Tokenizer

if TEST['Transformer']:
    ttnn.close_device(device)
    device = ttnn.open_device(device_id=device_id)
    
    x_torch = torch.randint(1, args.vocab_size - 1, (1, 152))
    seq_len = 1

    x = ttnn.from_torch(x_torch, device=device)
    x = ttnn.to_layout(x, layout=ttnn.ROW_MAJOR_LAYOUT)

    torch_trans = torchTransformer(args, state_dict)
    start = time.time()
    to_trans = torch_trans.forward(x_torch)
    print(f"torch: {time.time() - start:.3f}")

    tt_trans = Transformer(args, state_dict, device)
    start = time.time()
    o_trans = tt_trans.forward(x)
    print("Transformer:")
    print(f"ttnn: {time.time() - start:.3f}")

    check_close(to_trans.bfloat16(), o_trans)

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
torch: 2.647
Reshapes: 0.292
Rotatory emb: 0.008
Output reshape: 2.172
Reshapes: 0.015
Rotatory emb: 0.008
Output reshape: 0.042
Reshapes: 0.015
Rotatory emb: 0.008
Output reshape: 0.043
Reshapes: 0.036
Rotatory emb: 0.018
Output reshape: 0.062
Reshapes: 0.012
Rotatory emb: 0.008
Output reshape: 0.043
Reshapes: 0.012
Rotatory emb: 0.011
Output reshape: 0.045
Reshapes: 0.031
Rotatory emb: 0.011
Output reshape: 0.049
Reshapes: 0.020
Rotatory emb: 0.010
Output reshape: 0.049
Transformer:
ttnn: 7.434
PCC: 0.93997


### Generation
#### WIP: tokens/s really bad atm

In [12]:
def get_next_token(logits, temperature, top_k):
    logits = logits[:, -1, :] # crop to just the final time step

    if temperature == 0.0:
        # "sample" the single most likely index
        _, idx_next = torch.topk(logits, k=1, dim=-1)
    else:
        # pluck the logits at the final step and scale by desired temperature
        logits = logits / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = torch.functional.F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
    
    return idx_next

In [13]:
temperature = 1.0
top_k = 300
token_to_generate = 100

start = "Dream comes true!"
start_ids = enc.encode(start, bos=True, eos=False)
x = (torch.tensor(start_ids, dtype=torch.long)[None, ...])


In [14]:
torch_trans = torchTransformer(args, state_dict)

# [1,13]
x = (torch.tensor(start_ids, dtype=torch.long)[None, ...])

for _ in range(token_to_generate):
    # if the sequence context is growing too long we must crop it at block_size
    x = x if x.size(1) <= args.max_seq_len else x[:, -args.max_seq_len:]
    
    # forward the model to get the logits for the index in the sequence
    logits = torch_trans.forward(x)

    idx_next = get_next_token(logits, temperature, top_k)
    # append sampled index to the running sequence and continue
    x = torch.cat((x, idx_next), dim=1)


print(enc.decode(x[0].tolist()))

Dream comes true! Spese and sunon. He was in their story. He ais to O, exha and two. All p ponies' roll things. Wise and all the paws were harmless.
 There was peace, but it was. Wama is turned.”
 Grandpa, little Grandpa.
Soon, old, sad, they were. Grandnie too. It would be happy.
One particular night was from that morning routine.
Mom and Grand down


In [ ]:
ttnn.close_device(device)
device = ttnn.open_device(device_id=device_id)
# device.enable_program_cache()

tt_trans = Transformer(args, state_dict, device)

x = (torch.tensor(start_ids, dtype=torch.long)[None, ...])

time_forward = 0
for _ in range(token_to_generate):
    tt_x = ttnn.from_torch(x, device=device)
    tt_x = ttnn.to_layout(tt_x, layout=ttnn.ROW_MAJOR_LAYOUT)

    start = time.time()
    logits = tt_trans.forward(tt_x)
    time_forward += time.time() - start

    logits = ttnn.to_torch(logits)
    idx_next = get_next_token(logits, temperature, top_k)    
    x = torch.cat((x, idx_next), dim=1)


print(enc.decode(x[0].tolist()))

print(f"Execution time: {time_forward}")

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
Reshapes: 0.009
Rotatory emb: 0.005
Output reshape: 1.849
Reshapes: 0.007
Rotatory emb: 0.004
Output reshape: 0.028
Reshapes: 0.008
Rotatory emb: 0.004
Output reshape: 0.027
Reshapes: 0.007
Rotatory emb: 0.004
Output reshape: 0.027
Reshapes: 0.007
Rotatory emb: 0.005
Output reshape: 0.028
Reshapes: 0.007
Rotatory emb: 0.004
Output reshape: 0.027
Reshapes: 0.007
Rotatory emb: 0.005
Output reshape: 0.029
Reshapes: 0.007
Rotatory emb: 0.004
Output reshape: 0.027
Reshapes: 0.007
Rotatory emb: 0.005
Output reshape: 0.594
Reshapes: 0.007
Rotatory emb: 0.005
Output reshape: 0.029
Reshapes: 0.007
Rotatory emb: 0.004
Output reshape: 0